<a href="https://colab.research.google.com/github/narendra974/AIMLOPS_IISC/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical AI and MLOps : Assignment 3


### Download the dataset.

Download the dataset attached with the assignment and store it in a pandas dataframe `df`. You are free to change the names as you like. You can split the datasets using `train_test_split` function from the `scikit-learn` library.

**1st dataset:** (df) For problem 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

from keras.models import Sequential
from keras.layers import Dense,Activation, Input

In [ ]:

df = pd.read_csv('Iris.csv')
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


## Problem 1 (5 marks)


You have to design and implement neural network models for multi-class classification using both a Multi-Layer Perceptron (MLP) and a Convolutional Neural Network (CNN). The dataset you will be working with contains samples from multiple classes. You need to experiment with different activation functions and loss functions for both models to find the best combinations for this task.

Instructions:

1. Load the multi-class dataset.

2. Preprocess the dataset as needed, including data splitting and normalization.

3. **Implement an MLP model**

Implement an MLP model with the following specifications:

*   Input layer with an appropriate number of neurons based on the dataset's features.
*   At least one hidden layer with a flexible number of neurons (you can experiment with this).
*   An output layer with neurons corresponding to the number of classes in the dataset.

Train the MLP model using the following settings:
*   Use two different activation functions for the hidden layers (e.g., ReLU and Sigmoid).
*   Use at least two different loss functions (e.g., Cross-Entropy and Mean Squared Error).
*   Experiment with various hyperparameters like learning rate, batch size, and the number of hidden neurons.

For each combination of activation function and loss function, train the model and evaluate its performance on the validation set using appropriate metrics (e.g., accuracy, F1-score).

Report the following for each combination:
*   Accuracy on the validation set.
*   F1-score on the validation set.
*   Confusion Matrix.

4. **Implement a CNN model**

Implement a 2D CNN model with the following specifications:

*   Convolutional layers with appropriate filters and kernel sizes.
*   At least one fully connected (dense) layer.
*   An output layer with neurons corresponding to the number of classes in the dataset.

Train the CNN model using the following settings:
*   Use two different activation functions for the convolutional and dense layers (e.g., ReLU and Tanh).
*   Use at least two different loss functions (e.g., Cross-Entropy and Categorical Hinge Loss).
*   Experiment with various hyperparameters like learning rate, batch size, and the number of filters.

For each combination of activation function and loss function, train the model and evaluate its performance on the validation set using appropriate metrics (e.g., accuracy, F1-score).

Report the following for each combination:
*   Accuracy on the validation set.
*   F1-score on the validation set.
*   Confusion Matrix.

In [ ]:
print(df["Species"].unique())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [ ]:
df.loc[df["Species"]=="Iris-setosa","Species"]=0
df.loc[df["Species"]=="Iris-versicolor","Species"]=1
df.loc[df["Species"]=="Iris-virginica","Species"]=2

df=df.iloc[np.random.permutation(len(df))]
print(df.head())

      Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm Species
13    14            4.3           3.0            1.1           0.1       0
58    59            6.6           2.9            4.6           1.3       1
136  137            6.3           3.4            5.6           2.4       2
63    64            6.1           2.9            4.7           1.4       1
48    49            5.3           3.7            1.5           0.2       0


In [ ]:
X=df.iloc[:,1:5].values
y=df.iloc[:,5].values

X_normalized=normalize(X,axis=0)

#Creating 80% train, 20% test data

total_length=len(df)
train_length=int(0.8*total_length)
test_length=int(0.2*total_length)

X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=y[:train_length]
y_test=y[train_length:]

print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

y_train=np_utils.to_categorical(y_train,num_classes=3)
y_test=np_utils.to_categorical(y_test,num_classes=3)
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Length of train set x: 120 y: 120
Length of test set x: 30 y: 30


In [ ]:
def create_model(hidden_neurons, hidden_activation, loss_function, learning_rate, batch_size):
  model=Sequential()
  # model.add(Dense(1000,input_dim=4,activation='relu'))
  model.add(Input(shape=(4,)))
  model.add(Dense(hidden_neurons, activation=hidden_activation))
  model.add(Dense(3,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=10,verbose=1)



## Problem 2 (5 marks)


1.  Write an essay to explain the MLOps Lifecycle, including the integration of DevOps, DataOps, and ModelOps. Save it in a file named "README.md" ("README.txt")
2.   Create a public github repository.
3.   Commit the file containing the essay (in "step 1") to the main branch.
4.   Create and checkout to a new branch.
5.   Edit the "README.md" file and make some changes.
6.   Commit the changes to the new branch and send a pull request to the main branch.

Share the github repository url in the assignment.

Make sure to not make any changes to the repository after the due date. Penalty will be same as the earlier and the last edited time will be considered for it.



```
# Write your answers here
```

